In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
last_skywalker = np.array([0.98, 0.9, -.9])

In [5]:
user1 = np.array([0.9, .8, -.6])

In [6]:
(user1 * last_skywalker).sum()

2.1420000000000003

In [10]:
casablanca = np.array([-.99, -.3, .8])

In [11]:
np.dot(user1.T, casablanca)

-1.611